# Connect to Local Spark Cluster using Spark Context (running via Docker)

In [1]:
%set_env JAVA_HOME=/Users/akshayiyer/Library/Java/JavaVirtualMachines/jdk8u222-b10/Contents/Home

env: JAVA_HOME=/Users/akshayiyer/Library/Java/JavaVirtualMachines/jdk8u222-b10/Contents/Home


In [2]:
%env

{'USER': 'akshayiyer',
 'PATH': '/Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/share/dotnet:~/.dotnet/tools:/Library/Frameworks/Mono.framework/Versions/Current/Commands',
 'LOGNAME': 'akshayiyer',
 'SSH_AUTH_SOCK': '/private/tmp/com.apple.launchd.K10i8yzXra/Listeners',
 'HOME': '/Users/akshayiyer',
 'SHELL': '/bin/zsh',
 '__CF_USER_TEXT_ENCODING': '0x1F5:0x0:0x0',
 'TMPDIR': '/var/folders/jr/5tpy30ms4lx89w1ltcmjw1_w0000gn/T/',
 'XPC_SERVICE_NAME': '0',
 'XPC_FLAGS': '0x0',
 'OLDPWD': '/Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl',
 'PWD': '/Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl',
 'SHLVL': '1',
 'TERM_PROGRAM': 'vscode',
 'TERM_PROGRAM_VERSION': '1.41.1',
 'LANG': 'en_US.UTF-8',
 'COLORTERM': 'truecolor',
 'TERM': 'xterm-color',
 'VIRTUAL_ENV': '/Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl',
 'PS1': '(udacity-dend-capstone-etl) %n@%m %1~ %# ',
 '_': '/Users/akshayiyer/Dev/GitHub/uda

In [3]:
# https://elmiko.github.io/2018/08/05/attaching-notebooks-with-radanalytics.html
import pyspark
conf=pyspark.SparkConf().setMaster('spark://172.18.0.2:7077')
#conf=pyspark.SparkConf().setMaster('local')
sc = pyspark.SparkContext(conf=conf)

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.IllegalArgumentException: requirement failed: Can only call getServletHandlers on a running MetricsSystem
	at scala.Predef$.require(Predef.scala:224)
	at org.apache.spark.metrics.MetricsSystem.getServletHandlers(MetricsSystem.scala:91)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:516)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
	at sun.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
	at java.lang.reflect.Constructor.newInstance(Constructor.java:423)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gatewa

In [ ]:
sc

## Program to calculate Pi on Spark Cluster

In [ ]:
%%time
import random
num_samples = 100000000

def inside(p):     
  x, y = random.random(), random.random()
  return x*x + y*y < 1

count = sc.parallelize(range(0, num_samples)).filter(inside).count()
pi = 4 * count / num_samples

print(pi)

# Connect to Local Spark Cluster using Spark Session (running via Docker)

In [ ]:
%set_env JAVA_HOME=/Users/akshayiyer/Library/Java/JavaVirtualMachines/jdk8u222-b10/Contents/Home

In [ ]:
import configparser
from datetime import datetime
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, date_format

In [ ]:
config = configparser.ConfigParser()
config.read('/Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/.aws/access_keys.cfg')

os.environ['AWS_ACCESS_KEY_ID']=config.get('AWS','KEY')
os.environ['AWS_SECRET_ACCESS_KEY']=config.get('AWS','SECRET')

In [ ]:
def create_spark_session():
    spark = SparkSession \
        .builder \
        .master('spark://127.0.0.1:7077') \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0")\
        .getOrCreate() 
    return spark

In [ ]:
spark = create_spark_session()
spark

## Read JSON file into Spark dataframe 

In [ ]:
song_data_file_path = 's3a://udacity-dend/song_data/A/B/C/TRABCEI128F424C983.json'

song_df = spark.read.json(song_data_file_path)

In [ ]:
song_df.printSchema()

# Testing Delta Lake

In [1]:
%set_env JAVA_HOME=/Users/akshayiyer/Library/Java/JavaVirtualMachines/jdk8u222-b10/Contents/Home

env: JAVA_HOME=/Users/akshayiyer/Library/Java/JavaVirtualMachines/jdk8u222-b10/Contents/Home


In [2]:
from pyspark.sql import SparkSession

In [3]:
def create_spark_session(master):
    spark = SparkSession \
        .builder \
        .master(master) \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0,io.delta:delta-core_2.11:0.5.0")\
        .appName("test-delta-lake") \
        .getOrCreate() 
    return spark

In [4]:
spark = create_spark_session("spark://127.0.0.1:7077")

In [5]:
spark

In [6]:
filepath = "/Users/akshayiyer/Dev/GitHub/udacity-dend-capstone-etl/data/delta-table"

In [7]:
data = spark.range(0, 5)
data.write.format("delta").save(filepath)

In [8]:
df = spark.read.format("delta").load(filepath)
df.show()

+---+
| id|
+---+
|  1|
|  2|
|  4|
|  3|
|  0|
+---+



In [9]:
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save(filepath)

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 0).load(filepath)
df.show()

In [ ]:
df = spark.read.format("delta").option("versionAsOf", 1).load(filepath)
df.show()